In [2]:
# !pip install langchain-pinecone

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 670.5 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pinecone-client
    Found existing installation: pinecone-client 2.2.2
    Uninstalling pinecone-client-2.2.2:
      Successfully uninstalled pinecone-client-2.2.2


In [6]:
from dotenv import load_dotenv
import os

# 加载当前目录下的.env文件
# load_dotenv()
# load_dotenv(override=True) 会重新读取.env
load_dotenv(override=True)
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
print(pinecone_api_key)

b693c16e-9b74-48af-a4af-c314a72f85df


In [7]:
from pinecone import Pinecone, ServerlessSpec
# https://app.pinecone.io/
pc = Pinecone(api_key=pinecone_api_key)

In [13]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

path = "../../file/最后一个问题.txt"

loader = TextLoader(path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [14]:
# 最新的Embedding方式
# cloudflare_workersai
from langchain_community.embeddings.cloudflare_workersai import (
    CloudflareWorkersAIEmbeddings,
)

account_id = os.getenv('CF_ACCOUNT_ID')
api_token = os.getenv('CF_API_TOKEN')
embeddings = CloudflareWorkersAIEmbeddings(
    account_id=account_id,
    api_token=api_token,
    model_name="@cf/baai/bge-small-en-v1.5",
)

In [15]:
import time

index_name = "langchain-index"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [16]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(docs, embeddings, index_name=index_name)

In [17]:
query = "What did the president say about Ketanji Brown Jackson"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

Rachel, undeterred, held the book close, as if it were a talisman against the chaos outside. "We can't leave," she said, her voice steady. "We have to wait out the storm."

Clement, hesitating, was drawn into Rachel's gaze. Something in her eyes sparkled, like the half-remembered promise of a forgotten dream
